# <font color='red'>SEDES - Gerador de Dados</font>

# <font color='blue'>1.0 import</font>

In [1]:
import csv
import os
import pandas as pd
import random as rd
import datetime as dt
from dateutil.relativedelta import relativedelta

# <font color='blue'>2.0 parâmetros de ambiente</font>

In [22]:
amb_windows = False

HOME_PASTA = '' if amb_windows else os.path.expanduser("~")

caminho_in  = '../dados/apoio/'
caminho_out = '../dados/simulacao/'

In [3]:
# data inicial de simulação
DATA_INICIAL = (2018, 1, 10)

# qtd de meses para cada formulário
MESES_INC = 1


# municípios
municipios = {
    'resp_form': (0.6, 0.8) # intervalo de % de municípios que responderam ao formulário
}
profissionais = {
    'qtd_nomes': (1000, 20) # (numéro médio de nomes, variação positiva e negativa a ser sorteada)
}
formacoes = {
    'qtd_formacoes': (10, 15) # intervalo da quantidade de profissoes que participarão da simulação
}
cargos = {
    'qtd_cargos': (5, 8) # intervalo da quantidade de profissoes que participarão da simulação
}
horas_dedicadas = (8, 16, 24, 32, 40, 48) # possibilidades de horas dedicadas por mês
dedicacao_exclusiva = ((1, 0), (50, 50)) # pesos para sorteio de horas exclusivas (1, 0) 1="sim" e 0="não"
orgao_gestor = (1, 3) # possibilidades de orgão gestor por município a cada data de referência
fg_lei_reg_suas = ((1, 0), (60, 40)) # pesos para sorteio de lei de registro (1, 0) 1="sim" e 0="não"
fg_tem_PPAS = ((1, 0), (50, 50)) # pesos para sorteio de PPAS (1, 0) 1="sim" e 0="não"
DEF_fg_org_QQD = ((1, 0), (40, 60)) # pesos para sorteio de QQD (1, 0) 1="sim" e 0="não"
DEF_as_municipio_VS = ((1, 0), (75, 25)) # pesos para sorteio de as_municipio VS (1, 0) 1="sim" e 0="não"
DEF_as_municipio_equipe_VS = ((1, 0), (50, 50)) # pesos para sorteio de as_municipio equipe VS (1, 0) 1="sim" e 0="não"
DEF_as_municipio_instr_gestao = ((1, 0), (80, 20)) # pesos para sorteio de as_instr_gestão (1, 0) 1="sim" e 0="não"

# <font color='blue'>3.0 extração</font>

## 3.1 extracao de dados externo

In [6]:
# abrindo os arquivos...

arq_municipios =   caminho_in + "tab_municipios.csv"
arq_VS =           caminho_in + "tab_VS.csv"
arq_escolaridade = caminho_in + "tab_escolaridade.csv"
arq_formacao =     caminho_in + "tab_formacoes.csv"
arq_cargo =        caminho_in + "tab_cargos.csv"
arq_instr_gestao = caminho_in + "tab_instr_gestao.csv"

df_municipios =   pd.read_csv(arq_municipios,   sep = ',', dtype = str)
df_VS =           pd.read_csv(arq_VS,           sep = ',', dtype = str)
df_escolaridade = pd.read_csv(arq_escolaridade, sep = ',', dtype = str)
df_formacao =     pd.read_csv(arq_formacao,     sep = ',', dtype = str)
df_cargo =        pd.read_csv(arq_cargo,        sep = ',', dtype = str)
df_instr_gestao = pd.read_csv(arq_instr_gestao, sep = ',', dtype = str)

print("municipios:              linhas = %s , colunas = %s " % df_municipios.shape)
print("VS:                      linhas = %s , colunas = %s " % df_VS.shape)
print("escolaridade:            linhas = %s , colunas = %s " % df_escolaridade.shape)
print("formacoes:               linhas = %s , colunas = %s " % df_formacao.shape)
print("cargos:                  linhas = %s , colunas = %s " % df_cargo.shape)
print("instrumentos de gestão:  linhas = %s , colunas = %s " % df_instr_gestao.shape)

municipios:              linhas = 217 , colunas = 4 
VS:                      linhas = 8 , colunas = 1 
escolaridade:            linhas = 12 , colunas = 1 
formacoes:               linhas = 225 , colunas = 1 
cargos:                  linhas = 133 , colunas = 1 
instrumentos de gestão:  linhas = 5 , colunas = 1 


In [7]:
df_municipios['city_ibge_code'] = df_municipios['city_ibge_code'].astype('int64')

In [8]:
cols = ['city_ibge_code', 'estimated_population']
df_municipios[cols] = df_municipios[cols].astype('int64')

In [9]:
df_cargo['cargo'] = df_cargo['cargo'].str.strip()

# <font color='blue'>4.0 transformação</font>

## 4.1 tabelas principais

### 4.1.1 datas de formulário

In [10]:
# cria a sequência de datas de referência passada
dt_inicial = dt.date(DATA_INICIAL[0], DATA_INICIAL[1], DATA_INICIAL[2])
lst_datas = []
dt_atual = dt_inicial

while dt_atual < dt.date.today():
    lst_datas.append(dt_atual)
    dt_atual = dt_atual + relativedelta(months = MESES_INC)
df_datas = pd.DataFrame(lst_datas, columns=['dt_ref'])
# df_datas

### 4.1.2 municípios (tb_municipio)

In [11]:
# cria lista de orgãos gestores
lst_OG = ['Orgão Gestor ' + str(x + 1) for x in range(orgao_gestor[1])]

# cria a tb_municípios
df_tb_municipio = df_municipios[['city_ibge_code', 'city', 'estimated_population']]
df_tb_municipio.columns = ['PK_municipio', 'nome_municipio', 'pop_municipio']
df_tb_municipio['OGs'] = df_tb_municipio.apply(lambda x: rd.sample(lst_OG, k=rd.randint(orgao_gestor[0], 
                                                                                        orgao_gestor[1])), 
                                                                                        axis=1)
# df_tb_municipio

### 4.1.3 VS (tb_VS) (vigilância Socioassistencial)

In [12]:
df_tb_VS = df_VS.reset_index()
df_tb_VS.columns = ['PK_VS', 'nome_VS']
# df_tb_VS

### 4.1.4 instrumentos de gestão(tb_instr_gestao)

In [13]:
df_tb_instr_gestao = df_instr_gestao.reset_index()
df_tb_instr_gestao.columns = ['PK_instr_gestao', 'nome_instr_gestao']
# df_tb_instr_gestao

### 4.1.5 equipe VS (tb_equipe_VS)

In [14]:
# gera a lista total de profissionais que será usada
lst_tmp_prof = []
qtd_prof = rd.randint(profissionais['qtd_nomes'][0] - profissionais['qtd_nomes'][1], 
                      profissionais['qtd_nomes'][0] + profissionais['qtd_nomes'][1])
# lista de escolaridades que participarão da simulação
lst_escolaridade = df_escolaridade.values.tolist()

# lista de formações que participarão da simulação
k = int(rd.uniform(formacoes['qtd_formacoes'][0], formacoes['qtd_formacoes'][1]))
lst_formacao = rd.sample(df_formacao.values.tolist(), k = k)

# lista de cargos que participarão da simulação
k = int(rd.uniform(cargos['qtd_cargos'][0], cargos['qtd_cargos'][1]))
lst_cargo = rd.sample(df_cargo.values.tolist(), k = k)

for i in range(qtd_prof):
    lst_tmp_escolaridade = rd.sample(lst_escolaridade, k = 1)
    lst_tmp = ['Profissional ' + str(i + 1), 
               rd.sample(lst_escolaridade, k = 1)[0][0],
               rd.sample(lst_formacao, k = 1)[0][0],
               rd.sample(lst_cargo, k = 1)[0][0],
               rd.sample(horas_dedicadas, k = 1)[0],
               rd.choices(dedicacao_exclusiva[0], dedicacao_exclusiva[1], k=1)[0]
              ]
    lst_tmp_prof.append(lst_tmp)


cols = ['PK_prof_VS', 'nome_prof_VS', 'escolaridade_VS', 'formacao_VS', 'funcao_cargo_VS', 'horas_dedicadas_VS', 'exclusivo_VS']
df_tb_equipe_VS = pd.DataFrame(lst_tmp_prof).reset_index()
df_tb_equipe_VS.columns = cols
# df_tb_equipe_VS

## 4.2 tabelas de movimento

### 4.2.1 M_00 (Identificação do município) (tb_municipio_form)

#### 4.2.1.1 tb_municipio_form

In [15]:
# determina a % de municípios que responderam ao formulário
qtd_mun_resp = int(rd.uniform(municipios['resp_form'][0], municipios['resp_form'][1]) * df_tb_municipio.shape[0])

# determina quais os municipios responderam ao formulário
df_tb_municipio_form = df_tb_municipio[['PK_municipio', 'OGs']].sample(n=qtd_mun_resp)
# df_tb_municipio_form

In [16]:
df_datas['key'] = 1; df_tb_municipio_form['key'] = 1

df_tb_municipio_form = df_tb_municipio_form.merge(df_datas, on='key').drop('key', axis=1)
# tb_municipio_form = pd.merge(tb_municipio_form, df_datas, on ='key').drop("key", 1) 

In [17]:
# escolhe o orgão gestor da lista de possíveis da coluna 'OGs'
df_tb_municipio_form['orgao_gestor'] = df_tb_municipio_form['OGs'].apply(lambda x: x[rd.randint(0, len(x) - 1)])
df_tb_municipio_form.drop('OGs', axis=1, inplace=True)
df_tb_municipio_form['resp_registro'] = 'Secretarias Municipais de A.S. / SEDES' 
df_tb_municipio_form['freq_registro'] = 'Semestral'
df_tb_municipio_form['fg_lei_reg_suas'] = df_tb_municipio_form['PK_municipio'].apply(lambda x: rd.choices(fg_lei_reg_suas[0], fg_lei_reg_suas[1], k=1)[0])
df_tb_municipio_form['fg_tem_PPAS'] = df_tb_municipio_form['PK_municipio'].apply(lambda x: rd.choices(fg_tem_PPAS[0], fg_tem_PPAS[1], k=1)[0])
df_tb_municipio_form['fg_org_QQD'] = df_tb_municipio_form['PK_municipio'].apply(lambda x: rd.choices(DEF_fg_org_QQD[0], 
                                                                                                     DEF_fg_org_QQD[1], 
                                                                                                     k=1)[0])

df_tb_municipio_form.rename(columns={'PK_municipio': 'FK_municipio'}, inplace=True)
# df_tb_municipio_form


#### 4.2.1.2 as_municipio_VS

In [18]:
df_tb_VS['key'] = 1; df_tb_municipio_form['key'] = 1
df_tb_as_municipio_VS = df_tb_municipio_form.merge(df_tb_VS, on='key').drop(['key', 'nome_VS'], axis=1)
df_tb_as_municipio_VS['fica'] = df_tb_as_municipio_VS['FK_municipio'].apply(lambda x: rd.choices(DEF_as_municipio_VS[0], 
                                                                                                 DEF_as_municipio_VS[1], 
                                                                                                 k=1)[0])

cols = ['FK_municipio', 'dt_ref', 'PK_VS']
df_tb_as_municipio_VS = df_tb_as_municipio_VS[(df_tb_as_municipio_VS['fica'] == 1)][cols]
df_tb_as_municipio_VS.rename(columns={'PK_VS': 'FK_VS'}, inplace=True)
# df_tb_as_municipio_VS


#### 4.2.1.3 as_municipio_equipe_VS

In [19]:
df_tb_equipe_VS['key'] = 1; df_tb_municipio_form['key'] = 1
df_tb_as_municipio_equipe_VS = df_tb_municipio_form.merge(df_tb_equipe_VS, on='key')
df_tb_as_municipio_equipe_VS['fica'] = df_tb_as_municipio_VS['FK_municipio'].apply(lambda x: rd.choices(DEF_as_municipio_equipe_VS[0], 
                                                                                                        DEF_as_municipio_equipe_VS[1], 
                                                                                                        k=1)[0])
cols = ['FK_municipio', 'dt_ref', 'PK_prof_VS']
df_tb_as_municipio_equipe_VS = df_tb_as_municipio_equipe_VS[(df_tb_as_municipio_equipe_VS['fica'] == 1)][cols]
df_tb_as_municipio_equipe_VS.rename(columns={'PK_prof_VS': 'FK_prof_VS'}, inplace=True)
# df_tb_as_municipio_equipe_VS


#### 4.2.1.3 as_municipio_instr_gestao

In [20]:
df_tb_instr_gestao['key'] = 1; df_tb_municipio_form['key'] = 1
df_tb_as_municipio_instr_gestao = df_tb_municipio_form.merge(df_tb_instr_gestao, on='key')
df_tb_as_municipio_instr_gestao['fica'] = df_tb_as_municipio_instr_gestao['FK_municipio'].apply(lambda x: rd.choices(DEF_as_municipio_instr_gestao[0], 
                                                                                                                     DEF_as_municipio_instr_gestao[1], 
                                                                                                                     k=1)[0])
cols = ['FK_municipio', 'dt_ref', 'PK_instr_gestao']
df_tb_as_municipio_instr_gestao = df_tb_as_municipio_instr_gestao[(df_tb_as_municipio_instr_gestao['fica'] == 1)][cols]
df_tb_as_municipio_instr_gestao.rename(columns={'PK_instr_gestao': 'FK_instr_gestao'}, inplace=True)
# df_tb_as_municipio_instr_gestao


## 4.2 acertos finais

In [21]:
# drop de colunas usadas e não mais úteis
df_tb_municipio.drop('OGs', axis=1, inplace=True)
df_tb_VS.drop('key', axis=1, inplace=True)
df_tb_equipe_VS.drop('key', axis=1, inplace=True)
df_tb_instr_gestao.drop('key', axis=1, inplace=True)
df_tb_municipio_form.drop('key', axis=1, inplace=True)

# <font color='blue'>5.0 carga</font>

In [23]:
# grava tabelas
df_tb_municipio.to_csv(caminho_out + "tb_municipios.csv", index=False)
df_tb_VS.to_csv(caminho_out + "tb_VS.csv", index=False)
df_tb_equipe_VS.to_csv(caminho_out + "tb_equipe_VS.csv", index=False)
df_tb_instr_gestao.to_csv(caminho_out + "tb_instr_gestao.csv", index=False)

# grava associativas
df_tb_municipio_form.to_csv(caminho_out + "tb_municipio_form.csv", index=False)
df_tb_as_municipio_VS.to_csv(caminho_out + "as_municipio_VS.csv", index=False)
df_tb_as_municipio_equipe_VS.to_csv(caminho_out + "as_municipio_equipe_VS.csv", index=False)
df_tb_as_municipio_instr_gestao.to_csv(caminho_out + "as_municipio_instr_gestao.csv", index=False)


# TESTES